# Urdhva Thiryagbhyam Vedic Multiplication Benchmark
This notebook compares conventional multiplication with the Vedic Urdhva Thiryagbhyam method.

In [ ]:
import time
import random
import statistics
from typing import List, Tuple

class MultiplicationBenchmark:
    """
    A comprehensive benchmark class to compare conventional and Vedic multiplication methods.
    """
    def __init__(self):
        self.results = {
            'conventional': [],
            'vedic': [],
            'test_numbers': []
        }

    def conventional_multiplication(self, num1: int, num2: int) -> int:
        return num1 * num2

    def vedic_multiplication_urdhva_thiryagbhyam(self, num1: int, num2: int) -> int:
        # Handle sign
        sign = 1
        if (num1 < 0) ^ (num2 < 0):
            sign = -1
        num1, num2 = abs(num1), abs(num2)
        # Convert to strings and reverse for easier processing (right-to-left)
        str1 = str(num1)[::-1]
        str2 = str(num2)[::-1]
        # Initialize result array - maximum possible digits
        result = [0] * (len(str1) + len(str2))
        # Apply Urdhva Thiryagbhyam method
        # For each position in the result, we sum:
        # - All products where digit positions add up to current result position
        for i in range(len(str1)):
            for j in range(len(str2)):
                # Multiply digits and add to appropriate position
                digit_product = int(str1[i]) * int(str2[j])
                result[i + j] += digit_product
        # Handle carries from right to left
        carry = 0
        for i in range(len(result)):
            total = result[i] + carry
            result[i] = total % 10
            carry = total // 10
        # Convert result back to integer (reverse to get correct order)
        final_result = 0
        for i in range(len(result) - 1, -1, -1):
            if result[i] != 0 or final_result != 0:
                final_result = final_result * 10 + result[i]
        return final_result * sign if final_result != 0 else 0

    def time_function(self, func, num1: int, num2: int, iterations: int = 1000) -> float:
        """
        Time a multiplication function over multiple iterations.
        Args:
            func: Function to time
            num1: First number
            num2: Second number
            iterations: Number of iterations to run
        Returns:
            Average time per operation in seconds
        """
        start_time = time.perf_counter()
        for _ in range(iterations):
            result = func(num1, num2)
        end_time = time.perf_counter()
        return (end_time - start_time) / iterations

    def verify_correctness(self, test_cases: List[Tuple[int, int]]) -> bool:
        """
        Verify that Vedic multiplication produces correct results.
        Args:
            test_cases: List of (num1, num2) tuples to test
        Returns:
            True if all tests pass, False otherwise
        """
        print("Verifying correctness of Vedic multiplication...")
        all_correct = True
        failed_cases = []
        for i, (num1, num2) in enumerate(test_cases):
            conventional_result = self.conventional_multiplication(num1, num2)
            vedic_result = self.vedic_multiplication_urdhva_thiryagbhyam(num1, num2)
            if conventional_result != vedic_result:
                print(f"Test {i+1} FAILED: {num1} × {num2}")
                print(f"  Conventional: {conventional_result}")
                print(f"  Vedic: {vedic_result}")
                failed_cases.append((num1, num2))
                all_correct = False
        if all_correct:
            print(f" All {len(test_cases)} tests passed!")
        else:
            print(f" {len(failed_cases)} out of {len(test_cases)} tests failed!")
        return all_correct

    def run_benchmark(self, test_ranges: List[Tuple[int, int]], iterations: int = 1000):
        """
        Run comprehensive benchmark comparing multiplication methods.
        Args:
            test_ranges: List of (min, max) ranges for random number generation
            iterations: Number of iterations per test case
        """
        print("=" * 80)
        print("VEDIC MULTIPLICATION BENCHMARK")
        print("Comparing Conventional vs Urdhva Thiryagbhyam Methods")
        print("=" * 80)
        # Generate test cases
        test_cases = []
        for min_val, max_val in test_ranges:
            for _ in range(10):  # 10 random pairs per range
                num1 = random.randint(min_val, max_val)
                num2 = random.randint(min_val, max_val)
                test_cases.append((num1, num2))
        # Add some edge cases
        edge_cases = [
            (0, 123), (123, 0), (1, 999), (999, 1),
            (-123, 456), (123, -456), (-123, -456)
        ]
        test_cases.extend(edge_cases)
        # Verify correctness first
        print("\n" + "=" * 50)
        print("CORRECTNESS VERIFICATION")
        print("=" * 50)
        if not self.verify_correctness(test_cases[:30]):  # Test first 30 cases
            print(" Correctness verification failed!")
            return
        print("\n" + "=" * 50)
        print("PERFORMANCE BENCHMARKING")
        print("=" * 50)
        # Run performance tests
        overall_conventional_times = []
        overall_vedic_times = []
        for range_idx, (min_val, max_val) in enumerate(test_ranges):
            print(f"\n Testing Range {range_idx + 1}: {min_val} to {max_val}")
            print("-" * 40)
            # Generate test numbers for this range
            range_test_cases = [(random.randint(min_val, max_val), random.randint(min_val, max_val))
                               for _ in range(5)]
            conventional_times = []
            vedic_times = []
            for num1, num2 in range_test_cases:
                # Time conventional multiplication
                conv_time = self.time_function(self.conventional_multiplication, num1, num2, iterations)
                conventional_times.append(conv_time)
                # Time Vedic multiplication
                vedic_time = self.time_function(self.vedic_multiplication_urdhva_thiryagbhyam, num1, num2, iterations)
                vedic_times.append(vedic_time)
            # Calculate statistics for this range
            conv_avg = statistics.mean(conventional_times)
            vedic_avg = statistics.mean(vedic_times)
            overall_conventional_times.extend(conventional_times)
            overall_vedic_times.extend(vedic_times)
            print(f"Average times (per operation):")
            print(f"  Conventional:     {conv_avg:.8f} seconds")
            print(f"  Urdhva Thiryag:   {vedic_avg:.8f} seconds")
            # Performance comparison
            ratio = vedic_avg / conv_avg
            print(f"Performance ratio:  {ratio:.2f}x")
            if ratio < 1.0:
                print(f"Vedic is {1/ratio:.2f}x FASTER!")
            else:
                print(f"Vedic is {ratio:.2f}x slower")
        # Overall statistics
        print(f"\n" + "=" * 50)
        print("OVERALL PERFORMANCE SUMMARY")
        print("=" * 50)
        overall_conv_avg = statistics.mean(overall_conventional_times)
        overall_vedic_avg = statistics.mean(overall_vedic_times)
        overall_ratio = overall_vedic_avg / overall_conv_avg
        print(f"Overall average times:")
        print(f"  Conventional:     {overall_conv_avg:.8f} seconds")
        print(f"  Urdhva Thiryag:   {overall_vedic_avg:.8f} seconds")
        print(f"Overall ratio:      {overall_ratio:.2f}x")
        if overall_ratio < 1.0:
            print(f"Vedic method is {1/overall_ratio:.2f}x faster overall!")
        else:
            print(f"Vedic method is {overall_ratio:.2f}x slower overall")

    def demonstrate_methods(self):
        """
        Demonstrate both multiplication methods with examples.
        """
        print("\n" + "=" * 60)
        print("DEMONSTRATION OF MULTIPLICATION METHODS")
        print("=" * 60)
        examples = [
            (3, 7),      # Single digits
            (9, 8),      # Single digits
            (23, 45),    # Two digits
            (123, 456),  # Three digits
            (99, 99),    # Special case
            (12, 34)     # Two digits
        ]
        for num1, num2 in examples:
            print(f"\nExample: {num1} × {num2}")
            print("-" * 30)
            # Conventional
            conv_result = self.conventional_multiplication(num1, num2)
            print(f" Conventional result: {conv_result}")
            # Vedic
            vedic_result = self.vedic_multiplication_urdhva_thiryagbhyam(num1, num2)
            print(f" Vedic result: {vedic_result}")
            # Verify
            if conv_result == vedic_result:
                print("Both methods agree!")
            else:
                print("Methods disagree!")


In [ ]:
def main():
    """
    Main function to run the benchmark.
    """
    print("URDHVA THIRYAGBHYAM VEDIC MULTIPLICATION BENCHMARK")
    print("=" * 60)
    benchmark = MultiplicationBenchmark()
    # Demonstrate the methods
    benchmark.demonstrate_methods()
    # Define test ranges
    test_ranges = [
        (1, 9),            # Single digit numbers
        (10, 99),          # 2-digit numbers
        (100, 999),        # 3-digit numbers
        (1000, 9999),      # 4-digit numbers
        (10000, 99999),    # 5-digit numbers
    ]
    # Run benchmark
    benchmark.run_benchmark(test_ranges, iterations=10000)
    print("\n" + "=" * 80)
    print("ANALYSIS SUMMARY")
    print("=" * 80)
    print("""
OBSERVATIONS:
1. CONVENTIONAL MULTIPLICATION:
   - Uses Python's built-in multiplication operator (*)
   - Highly optimized at the C language level
   - Consistently fast across all number ranges
   - Handles all edge cases automatically
2. URDHVA THIRYAGBHYAM VEDIC METHOD:
   - Implements the traditional "Vertical and Crosswise" technique
   - Works by multiplying digits in a specific pattern
   - Requires manual handling of carries and string operations
   - More computational steps due to digit-by-digit processing
    """)

if __name__ == "__main__":
    main()
